## Preprocessing

### 0. 사용 라이브러리

In [208]:
import pandas as pd
import numpy as np

from konlpy.tag import Okt

# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install git+https://github.com/ssut/py-hanspell.git

from pykospacing import Spacing
from hanspell import spell_checker

### 1. 데이터 불러오기

In [209]:
# 데이터 불러오기

train_data = pd.read_table('./data/ratings_train.txt')
test_data = pd.read_table('./data/ratings_test.txt')


In [210]:
train_data.info() # 150000 row

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [211]:
test_data.info() # 50000 row

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


### 2. 데이터 정제

#### 2.1 중복값 처리

In [212]:
# train_data 중복값 확인

print('id 중복값: ', train_data['id'].notnull().sum() - len(train_data['id'].unique()))
print('document 중복값: ', train_data['document'].notnull().sum() - len(train_data['document'].unique()))

print('label 값: ', train_data['label'].unique()) # 0, 1

id 중복값:  0
document 중복값:  3812
label 값:  [0 1]


In [213]:
# test_data 중복값 확인

print('id 중복값: ', test_data['id'].notnull().sum() - len(test_data['id'].unique()))
print('document 중복값: ', test_data['document'].notnull().sum() - len(test_data['document'].unique()))

print('label 값: ', test_data['label'].unique()) # 0, 1

id 중복값:  0
document 중복값:  839
label 값:  [1 0]


In [214]:
# 중복값 제거

train_data.drop_duplicates(['document'], inplace=True)
test_data.drop_duplicates(['document'], inplace=True)

print(len(train_data), len(test_data))

146183 49158


#### 2.2 구두점, 특수문자 제거

In [215]:
train_data['document'] = train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','') # [ㄱ - ㅎ, ㅏ - ㅣ, 가 - 핳] 제외 제거
train_data['document'] = train_data['document'].str.replace('^ +', '') # 화이트 스페이스 제거

test_data['document'] = test_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_data['document'] = test_data['document'].str.replace('^ +', '')

<ipython-input-215-152f18873e90>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','') # [ㄱ - ㅎ, ㅏ - ㅣ, 가 - 핳] 제외 제거
<ipython-input-215-152f18873e90>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', '') # 화이트 스페이스 제거
<ipython-input-215-152f18873e90>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
<ipython-input-215-152f18873e90>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', '')


#### 2.3 결측치 처리

In [216]:
# 결측치 확인

print(len(train_data[train_data['document']==""]), train_data['document'].isnull().sum())
train_data.replace("", np.nan, inplace=True) # 비어있는 값 null로 변환
train_data[train_data['document'].isnull()==True]

789 1


,id,document,label
404,4221289,NaN,0
412,9509970,NaN,1
470,10147571,NaN,1
584,7117896,NaN,0
593,6478189,NaN,0
...,...,...,...
149309,6715725,NaN,1
149358,6780491,NaN,0
149364,8014701,NaN,1
149630,3508604,NaN,0


In [217]:
# 결측치 확인

print(len(test_data[test_data['document']==""]), test_data['document'].isnull().sum())
test_data.replace("", np.nan, inplace=True) # 비어있는 값 null로 변환
test_data[test_data['document'].isnull()==True]

305 1


,id,document,label
1,9274899,NaN,0
116,6910486,NaN,1
254,4976468,NaN,0
468,7517102,NaN,0
504,2096424,NaN,0
...,...,...,...
49420,5187147,NaN,0
49459,6381245,NaN,1
49803,5309713,NaN,1
49871,9767991,NaN,0


In [218]:
# 결측치 제거

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [219]:
# 결측치 확인

print(train_data['document'].isnull().sum(), test_data['document'].isnull().sum())
print(len(train_data), len(test_data))

0 0
145393 48852


### 3. 띄어쓰기, 맞춤법

In [220]:
spacing = Spacing()

test_sentence = train_data.iloc[1, 1]
spaced_sentence = spacing(test_sentence)
print(test_sentence)
print(spaced_sentence)

흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
흠포스터 보고 초딩 영화 줄오버 연기조차 가볍지 않구나


In [221]:
test_sentence = train_data.iloc[2, 1]
spaced_sentence = spacing(test_sentence)
print(test_sentence)
print(spaced_sentence)

너무재밓었다그래서보는것을추천한다
너무 재밓었다 그래서 보는 것을 추천한다


In [222]:
test_sentence = train_data.iloc[10, 1]
spaced_sentence = spacing(test_sentence)
print(test_sentence)
print(spaced_sentence)

걍인피니트가짱이다진짜짱이다
걍 인피니트가 짱이 다 진짜 짱이다


In [223]:
train_data['document'] = train_data['document'].apply(spacing)


In [224]:
train_data.iloc[10]

id                      9008700
document    걍 인피니트가 짱이 다 진짜 짱이다
label                         1
Name: 10, dtype: object

In [227]:
spaced_train_data = train_data.copy()
spaced_train_data.to_csv('./data/ratings_train_spaced.csv', index=False)

### 3. 토큰화

형태소 분석기 종류
Okt(Open Korea Text), 메캅(Mecab), 코모란(Komoran), 한나눔(Hannanum), 꼬꼬마(Kkma)

#### 3.1 Okt(Open Korea Text)

In [230]:
test_sentence = train_data.iloc[10, 1]

In [231]:
from konlpy.tag import Okt

okt = Okt()

print('OKT 형태소 분석 :',okt.morphs(test_sentence))
print('OKT 품사 태깅 :',okt.pos(test_sentence))
print('OKT 명사 추출 :',okt.nouns(test_sentence)) 

OKT 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이다']
OKT 품사 태깅 : [('걍', 'Adverb'), ('인피니트', 'Noun'), ('가', 'Josa'), ('짱', 'Noun'), ('이', 'Josa'), ('다', 'Adverb'), ('진짜', 'Noun'), ('짱', 'Noun'), ('이다', 'Josa')]
OKT 명사 추출 : ['인피니트', '짱', '진짜', '짱']


#### 3.2 꼬꼬마 Kkma

In [232]:
from konlpy.tag import Kkma

kkma = Kkma()

print('꼬꼬마 형태소 분석 :',kkma.morphs(test_sentence))
print('꼬꼬마 품사 태깅 :',kkma.pos(test_sentence))
print('꼬꼬마 명사 추출 :',kkma.nouns(test_sentence))  

꼬꼬마 형태소 분석 : ['걍', '인피', '니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
꼬꼬마 품사 태깅 : [('걍', 'MAG'), ('인피', 'NNG'), ('니트', 'NNG'), ('가', 'JKS'), ('짱', 'NNG'), ('이', 'JKS'), ('다', 'MAG'), ('진짜', 'MAG'), ('짱', 'NNG'), ('이', 'VCP'), ('다', 'EFN')]
꼬꼬마 명사 추출 : ['인피', '인피니트', '니트', '짱']


#### 3.3 은전한닢 형태소 분석기 mecab

https://eunjeon.blogspot.com/

https://cleancode-ws.tistory.com/97

https://m.blog.naver.com/PostView.nhn?blogId=aul-_-&logNo=221557243190&proxyReferer=https:%2F%2Fwww.google.com%2F

In [229]:
!pip install eunjeon

  Using cached eunjeon-0.4.0.tar.gz (34.7 MB)
  Running setup.py clean for eunjeon
Failed to build eunjeon
    Running setup.py install for eunjeon: started
    Running setup.py install for eunjeon: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Huitaek\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Huitaek\\AppData\\Local\\Temp\\pip-install-2n75i9jg\\eunjeon_8fe10e9d8300432f86de8780ee9bb898\\setup.py'"'"'; __file__='"'"'C:\\Users\\Huitaek\\AppData\\Local\\Temp\\pip-install-2n75i9jg\\eunjeon_8fe10e9d8300432f86de8780ee9bb898\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Huitaek\AppData\Local\Temp\pip-wheel-xbdqnbrk'
       cwd: C:\Users\Huitaek\AppData\Local\Temp\pip-install-2n75i9jg\eunjeon_8fe10e9d8300432f86de8780ee9bb898\
  Complete output (44 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\eunjeon
  copying eunjeon\constants.py -> build\lib.win-amd64-3.8\

### 불용어

In [46]:
# 불용어 사전 불러오기
stopwords = pd.read_csv("./data/stopwords.csv", encoding='CP949')
stopwords = list(stopwords['stopwords'])

In [30]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']